In [ ]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import numpy as np

In [ ]:
df = pd.read_csv('data/user_records_202503130422.csv')

In [ ]:
character_name_map = {0:"무작위",1:"재키",2:"아야",3:"피오라",4:"매그너스",5:"자히르",6:"나딘",7:"현우",8:"하트",9:"아이솔",10:"리 다이린",11:"유키",12:"혜진",13:"쇼우",14:"키아라",15:"시셀라",16:"실비아",17:"아드리아나",18:"쇼이치",19:"엠마",20:"레녹스",21:"로지",22:"루크",23:"캐시",24:"아델라",25:"버니스",26:"바바라",27:"알렉스",28:"수아",29:"레온",30:"일레븐",31:"리오",32:"윌리엄",33:"니키",34:"나타폰",35:"얀",36:"이바",37:"다니엘",38:"제니",39:"카밀로",40:"클로에",41:"요한",42:"비앙카",43:"셀린",44:"에키온",45:"마이",46:"에이든",47:"라우라",48:"띠아",49:"펠릭스",50:"엘레나",51:"프리야",52:"아디나",53:"마커스",54:"칼라",55:"에스텔",56:"피올로",57:"마르티나",58:"헤이즈",59:"아이작",60:"타지아",61:"이렘",62:"테오도르",63:"이안",64:"바냐",65:"데비&마를렌",66:"아르다",67:"아비게일",68:"알론소",69:"레니",70:"츠바메",71:"케네스",72:"카티야",73:"샬럿",74:"다르코",75:"르노어",76:"가넷",77:"유민",78:"히스이",79:"유스티나",9999:"나쟈"}

In [ ]:
character_id = 73

df_filtered = df[
#    (df['character_id'] == character_id) &
    (df['mode'].isin([2, 3])) & 
    (df['play_time'] >= (15*60)) & 
    (df['play_time'] <= (1350)) &
    (df['rp'] >= 5000)
]

df_standardized = pd.DataFrame({
    'damage_dealt_per_min': df_filtered['damage_dealt_to_players'] / (df_filtered['play_time'] / 60),
    'damage_taken_per_min': df_filtered['damage_taken_from_players'] / (df_filtered['play_time'] / 60),
    'healing_per_min': df_filtered['healing_amount'] / (df_filtered['play_time'] / 60),
    'weapon_id': np.where(df_filtered['character_id'] == character_id, 1, 0)
})

df_sampled = df_standardized.sample(frac=0.01, random_state=42)
df_sampled = df_sampled.sort_values(by='weapon_id', ascending=True)

print("샘플링된 데이터:")
print(df_sampled.head())
print(f"원본 데이터 크기: {len(df_filtered)}, 샘플링된 데이터 크기: {len(df_sampled)} ({round(len(df_sampled)/len(df_filtered)*100)}%)")

print("\n기본 통계:")
print(df_sampled.describe())

In [ ]:
fig = make_subplots(rows=1, cols=3, subplot_titles=["딜탱", "딜힐", "탱힐"])

# X-Y 그래프
fig.add_trace(
    go.Scatter(
        x=df_sampled['damage_dealt_per_min'],
        y=df_sampled['damage_taken_per_min'],
        mode='markers',
        marker=dict(size=6, opacity=0.5, color=df_sampled['weapon_id'], colorscale='inferno'),  # 색상을 weapon_id 기준으로
        name="딜탱",
        showlegend=False
    ),
    row=1, col=1
)

# X-Z 그래프
fig.add_trace(
    go.Scatter(
        x=df_sampled['damage_dealt_per_min'],
        y=df_sampled['healing_per_min'],
        mode='markers',
        marker=dict(size=6, opacity=0.5, color=df_sampled['weapon_id'], colorscale='inferno'),
        name="딜힐",
        showlegend=False
    ),
    row=1, col=2
)

# Y-Z 그래프
fig.add_trace(
    go.Scatter(
        x=df_sampled['damage_taken_per_min'],
        y=df_sampled['healing_per_min'],
        mode='markers',
        marker=dict(size=6, opacity=0.5, color=df_sampled['weapon_id'], colorscale='inferno', showscale=True),
        name="탱힐",
        showlegend=False
    ),
    row=1, col=3
)


fig.update_xaxes(range=[0, 3000], row=1, col=1)
fig.update_yaxes(range=[0, 3000], row=1, col=1)

fig.update_xaxes(range=[0, 3000], row=1, col=2)
fig.update_yaxes(range=[0, 3000], row=1, col=2)

fig.update_xaxes(range=[0, 3000], row=1, col=3)
fig.update_yaxes(range=[0, 3000], row=1, col=3)

# 레이아웃 설정
fig.update_layout(
    height=400, width=900,  # 그래프 크기 조정
    title_text=f'{character_name_map[character_id]} 딜탱힐 시각화 ({round(len(df_sampled)/len(df_filtered)*100)}%)'
)
# 그래프 표시
fig.show()